# Image_Classification_Pytorch (Inference)

<a class="anchor" id="0"></a>
# Table of Contents

1. [套件安裝與載入](#1)
1. [環境檢測與設定](#2)
1. [開發參數設定](#3)
1. [資料處理](#4)
    -  [載入CSV檔](#4.1)
    -  [檢查CSV檔缺失值](#4.2)
1. [定義模型方法](#5)
1. [製作資料集＆資料擴增＆推論模型](#6)
1. [待辦事項](#7)

# 1. 套件安裝與載入<a class="anchor" id="1"></a>
[Back to Table of Contents](#0)

In [1]:
!pip3 install git+https://github.com/rwightman/pytorch-image-models.git

  Cloning https://github.com/rwightman/pytorch-image-models.git to /tmp/pip-req-build-p188lkrz
  Running command git clone -q https://github.com/rwightman/pytorch-image-models.git /tmp/pip-req-build-p188lkrz


In [2]:
# 資料處理套件
import os
import gc
import cv2
import sys
import time
import timm
import random
import numpy as np
import pandas as pd
import albumentations as A

from tqdm import tqdm
from albumentations.pytorch.transforms import ToTensorV2

import warnings
warnings.filterwarnings("ignore")

In [3]:
# pytorch深度學習模組套件
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torchvision import models

# 2. 環境檢測與設定<a class="anchor" id="2"></a>
[Back to Table of Contents](#0)

In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

In [5]:
# 查看pytorch版本
print(torch.__version__)

1.7.0


In [6]:
'''執行環境參數設定'''

# (Boolean)是否為本機
LOCAL = False

# (Boolean)是否為 Colab
COLAB = False


'''檔案路徑參數設定'''

# (String)Root路徑
if LOCAL:
    PATH = r'../'
elif COLAB:
    PATH = r'/content/drive/My Drive/Colab Notebooks/'
else:
    PATH = r'../input/'
    
# (String)資料根路徑
DATA_ROOT_PATH = PATH+r'resized-2015-2019-diabetic-retinopathy-detection/' 

# (String)CSV根路徑
CSV_ROOT_PATH = PATH+r'aptos2019-blindness-detection/' 

# (String)測試資料路徑
TEST_DATA_PATH = DATA_ROOT_PATH+r'resized_test19/'

# (String)測試CSV路徑
TEST_CSV_PATH = CSV_ROOT_PATH+r'sample_submission.csv'

# (Boolean)是否要匯入Library
IMPORT_PYTORCH_LIBRARY = False

# (String)Library的路徑
PYTORCH_LIBRARY_PATH = PATH + "PyTorch_Library/"

# (String)讀取預訓練模型/權重的名稱，當fold model時，後面會自動加_NUMBER
LOAD_MODEL_NAME = ['efficientnet_v2_b0', 'efficientnet_v2_b0', 'efficientnet_v2_b0']

# (String)讀取預訓練模型/權重的儲存路徑
LOAD_MODEL_PATH = [PATH + r'test1111/', 
                   PATH + r'test1111/', 
                   PATH + r'test1111/']

In [7]:
if DEVICE != torch.device("cpu"):
    !nvidia-smi

Tue Jun  8 09:27:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
if not LOCAL and COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
if IMPORT_PYTORCH_LIBRARY:
    sys.path.append(PYTORCH_LIBRARY_PATH + "Custom_Loss.py")
    sys.path.append(PYTORCH_LIBRARY_PATH + "Custom_Model.py")

# 3. 開發參數設定<a class="anchor" id="3"></a>
[Back to Table of Contents](#0)

In [9]:
'''客製參數設定'''


'''資料參數設定'''

# (Int)分類數量
CLASSES = 5

# (Int)集成模型數量
ENSEMBLE_MODEL_COUNT = 3

# (Int List)有CSV檔該參數才有用，1則為不做交叉驗證
FOLD = [1]*ENSEMBLE_MODEL_COUNT

# (Int List)圖片尺寸
IMAGE_SIZE = [224]*ENSEMBLE_MODEL_COUNT

# (String)圖片副檔名
IMAGE_NAME_EXTENSION = '.jpg'

# (String)CSV圖片檔名欄位
IMAGE_NAME = 'id_code'

# (Boolean)CSV圖片檔名欄位是否包含副檔名
IMAGE_NAME_HAVE_EXTENSION = False

#  (Boolean)圖像轉為RGB
COLOR_CONVERT_RGB = True

# (Int)不同的種子會產生不同的Random或分層K-FOLD分裂, 42則是預設固定種子
SEED = 42
    
# (Boolean)如為True每次返回的卷積算法將是確定的，即默認算法
CUDNN_DETERMINISTIC = True

# (Boolean)PyTorch 中對模型裡的卷積層進行預先的優化，也就是在每一個卷積層中測試 cuDNN 提供的所有卷積實現算法，
# 然後選擇最快的那個。這樣在模型啟動的時候，只要額外多花一點點預處理時間，就可以較大幅度地減少訓練時間
CUDNN_BENCHMARK = True


'''資料擴增參數設定

資料擴增教學
https://zhuanlan.zhihu.com/p/107399127

資料擴增Doc
https://vfdev-5-albumentations.readthedocs.io/en/docs_pytorch_fix/api/augmentations.html
'''

# (Float)訓練集資料擴增的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_TEST_TRANSFORMS = [1.0]*ENSEMBLE_MODEL_COUNT

# (Float)模糊的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_BLUR = [0]*ENSEMBLE_MODEL_COUNT

# (Int)模糊的上限
BLUR_LIMIT = [3]*ENSEMBLE_MODEL_COUNT

# (Float)水平翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_HORIZONTALFLIP = [0]*ENSEMBLE_MODEL_COUNT

# (Float)垂直翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_VERTICALFLIP = [0]*ENSEMBLE_MODEL_COUNT

# (Float)水平和垂直翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_FLIP = [0]*ENSEMBLE_MODEL_COUNT

# (Float)隨機旋轉90度的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RANDOMROTATE90 = [0]*ENSEMBLE_MODEL_COUNT

# (Float)平移縮放旋轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_SHIFTSCALEROTATE = [0]*ENSEMBLE_MODEL_COUNT

# (Float)平移縮放旋轉的平移上限
SHIFTSCALEROTATE_SHIFT_LIMIT = [0.0625]*ENSEMBLE_MODEL_COUNT

# (Float)平移縮放旋轉的縮放上限
SHIFTSCALEROTATE_SCALE_LIMIT = [0.1]*ENSEMBLE_MODEL_COUNT

# (Float)平移縮放旋轉的旋轉上限
SHIFTSCALEROTATE_ROTATE_LIMIT = [45]*ENSEMBLE_MODEL_COUNT

# (Float)彈性變換的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_ELATICTRANSFORM = [0]*ENSEMBLE_MODEL_COUNT

# (Float)彈性變換的alpha高斯過濾參數
ELATICTRANSFORM_ALPHA = [1]*ENSEMBLE_MODEL_COUNT

# (Float)彈性變換的sigma高斯過濾參數
ELATICTRANSFORM_SIGMA = [50]*ENSEMBLE_MODEL_COUNT

# (Float)彈性變換的alpha_affine，範圍為（-alpha_affine，alpha_affine）
ELATICTRANSFORM_ALPHA_AFFINE = [50]*ENSEMBLE_MODEL_COUNT

# (Float)網格失真的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_GRIDDISTORTION = [0]*ENSEMBLE_MODEL_COUNT

# (Int)網格失真的每一條邊上網格單元數量
GRIDDISTORTION_NUM_STEPS = [5]*ENSEMBLE_MODEL_COUNT

# (Float)隨機亮度對比度的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RANDOMBRIGHTNESSCONTRAST_CONTRAST = [0]*ENSEMBLE_MODEL_COUNT

# (Float)隨機亮度的上限
RANDOMBRIGHTNESSCONTRAST_BRIGHTNESS_LIMIT = [0.2]*ENSEMBLE_MODEL_COUNT

# (Float)隨機對比度的上限
RANDOMBRIGHTNESSCONTRAST_CONTRAST_LIMIT = [0.2]*ENSEMBLE_MODEL_COUNT

# (Float)隨機色調飽和度的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_HUESATURATIONVALUE = [0]*ENSEMBLE_MODEL_COUNT

# (Float)隨機色調飽和度的色調上限
HUESATURATIONVALUE_HUE_SHIFT_LIMIT = [20]*ENSEMBLE_MODEL_COUNT

# (Float)隨機色調飽和度的飽和度上限
HUESATURATIONVALUE_SAT_SHIFT_LIMIT = [30]*ENSEMBLE_MODEL_COUNT

# (Float)隨機色調飽和度的值上限
HUESATURATIONVALUE_VAL_SHIFT_LIMIT = [20]*ENSEMBLE_MODEL_COUNT

# (Float)對比度受限自適應直方圖均衡的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_CLAHE = [0]*ENSEMBLE_MODEL_COUNT

# (Float)對比度受限自適應直方圖均衡的對比度上限
CLAHE_CLIP_LIMIT = [4.0]*ENSEMBLE_MODEL_COUNT

# (Float)隨機在圖像上生成黑色矩形的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_COARSEDROPOUT = [0]*ENSEMBLE_MODEL_COUNT

# (Int)隨機在圖像上生成黑色矩形的數量
COARSEDROPOUT_NUM_HOLES = [8]*ENSEMBLE_MODEL_COUNT

# (Int)隨機在圖像上生成黑色矩形的最大高度
COARSEDROPOUT_MAX_H_SIZE = [8]*ENSEMBLE_MODEL_COUNT

# (Int)隨機在圖像上生成黑色矩形的最大寬度
COARSEDROPOUT_MAX_W_SIZE = [8]*ENSEMBLE_MODEL_COUNT

# (Float)隨機縮放剪裁的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RANDOMRESIZEDCROP = [0]*ENSEMBLE_MODEL_COUNT

# (Float Tuple)隨機縮放剪裁之前的圖像比例縮放
RANDOMRESIZEDCROP_SCALE = [(0.08, 1.0)]*ENSEMBLE_MODEL_COUNT

# (Int)隨機縮放剪裁之前的圖像高度
RANDOMRESIZEDCROP_HEIGHT = IMAGE_SIZE

# (Int)隨機縮放剪裁之前的圖像寬度
RANDOMRESIZEDCROP_WIDTH = IMAGE_SIZE

# (Float)縮放的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RESIZE = [1.0]*ENSEMBLE_MODEL_COUNT

# (Int)縮放後的圖片高度
RESIZE_HEIGHT = IMAGE_SIZE

# (Int)縮放後的圖片寬度
RESIZE_WIDTH = IMAGE_SIZE

# (Float)正規化的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_NORMALIZE = [1.0]*ENSEMBLE_MODEL_COUNT

# (Float List)正規化的平均值([0,1]的參考平均值:[0.485, 0.456, 0.406], [-1,1]的參考平均值:[0.5, 0.5, 0.5])
NORMALIZE_MEAN = [[0.485, 0.456, 0.406]]*ENSEMBLE_MODEL_COUNT

# (Float List)正規化的標準差([0,1]的參考標準差[0.229, 0.224, 0.225], [-1,1]的參考標準差[0.5, 0.5, 0.5])
NORMALIZE_STD = [[0.229, 0.224, 0.225]]*ENSEMBLE_MODEL_COUNT

# (Float)正規化的PIXEL最大值(參考PIXEL最大值255.0)
NORMALIZE_MAX_PIXEL_VALUE = [255.0]*ENSEMBLE_MODEL_COUNT

# (Float)歸一化的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
# ToTensorV2()將[0, 255]的PIL.Image或[H, W, C]的numpy.ndarray數據，
# 轉換為形狀[C, H, W]的torch.FloadTensor，並歸一化。
P_TOTENSORV2 = [1.0]*ENSEMBLE_MODEL_COUNT


''''模型參數設定'''

# (String List)模型載入方式 - 1 MODEL;2 WEIGHT_OF_CUSTOM_MODEL;
# 3 WEIGHT_OF_TIMM_MODEL;4 WEIGHT_OF_BASE_MODEL
MODEL_LIST = [3]*ENSEMBLE_MODEL_COUNT

if 2 in MODEL_LIST:
    # (Model List)模型載入方式有CUSTOM_MODEL，依照index位置填入
    CUSTOM_MODEL = [None]*ENSEMBLE_MODEL_COUNT

if 3 in MODEL_LIST:
    # (Model List)模型載入方式有TIMM_MODEL，依照index位置填入
    TIMM_MODEL = ["tf_efficientnetv2_b0"]*ENSEMBLE_MODEL_COUNT

if 4 in MODEL_LIST:
    # (Model List)模型載入方式有BASE_MODEL，依照index位置填入
    BASE_MODEL = [None]*ENSEMBLE_MODEL_COUNT

# (Boolean)模型是否使用分類層輸出，否則為全連接層
CLASSIFIER_OUTPUT = [True]*ENSEMBLE_MODEL_COUNT

# (Float)Dropout比率
DROPOUT = [1]*ENSEMBLE_MODEL_COUNT

# (Boolean)Bias偏移量
BIAS = [True]*ENSEMBLE_MODEL_COUNT

# (Boolean)是否印出完整模型
MODEL_PRINT = [False]*ENSEMBLE_MODEL_COUNT


''''訓練參數設定'''

# (Int List)每批推論的尺寸
BATCH_SIZE = [256]*ENSEMBLE_MODEL_COUNT

In [10]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = CUDNN_DETERMINISTIC
    torch.backends.cudnn.benchmark = CUDNN_BENCHMARK

seed_everything(SEED)

# 4. 資料處理<a class="anchor" id="4"></a>
[Back to Table of Contents](#0)

## 4.1 載入CSV檔 <a class="anchor" id="4.1"></a>
[Back to Table of Contents](#0)

In [11]:
print('Reading data...')

# 讀取訓練資料集CSV檔
test_csv = pd.read_csv(TEST_CSV_PATH,encoding="utf8")

print('Reading data completed')

Reading data...
Reading data completed


In [12]:
# 顯示訓練資料集CSV檔
test_csv.head()

,id_code,diagnosis
0,0005cfc8afb6,0
1,003f0afdcd15,0
2,006efc72b638,0
3,00836aaacf06,0
4,009245722fa4,0


In [13]:
print("Shape of train_data :", test_csv.shape)

Shape of train_data : (1928, 2)


# 5. 定義模型方法<a class="anchor" id="5"></a>
[Back to Table of Contents](#0)

In [14]:
class build_model(nn.Module):

    def __init__(self, count, load_model_path):
        super().__init__()
        
        if MODEL_LIST[count] == 1:
            # 載入預訓練模型
            self.model = torch.load(load_model_path)
        elif MODEL_LIST[count] == 2:
            # 載入模型架構
            self.model = CUSTOM_MODEL[count]
        elif MODEL_LIST[count] == 3:
            self.model = timm.create_model(TIMM_MODEL[count], pretrained = None)
        elif MODEL_LIST[count] == 4:
            self.model = BASE_MODEL[count](pretrained = None)

        if CLASSIFIER_OUTPUT[count]:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Sequential(nn.Dropout(DROPOUT[count]), nn.Linear(n_features, CLASSES, bias = BIAS[count]))
        else:
            n_features = self.model.fc.in_features
            self.model.fc = nn.Sequential(nn.Dropout(DROPOUT[count]), nn.Linear(n_features, CLASSES, bias = BIAS[count]))

        if MODEL_LIST[count] != 1:
            # 載入預訓練權重
            self.model.load_state_dict(torch.load(load_model_path))
            
    def forward(self, x):
        x = self.model(x)
        return x

# 6. 製作資料集＆資料擴增＆推論模型<a class="anchor" id="6"></a>
[Back to Table of Contents](#0)

In [15]:
class MyDataset(Dataset):
    def __init__(self, df, transforms = None):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index: int):
        image_name = self.df[IMAGE_NAME].values[index]

        if IMAGE_NAME_HAVE_EXTENSION:
            image_path = TEST_DATA_PATH + image_name
        else:
            image_path = TEST_DATA_PATH + image_name + IMAGE_NAME_EXTENSION
            
        image = cv2.imread(image_path)
        
        if COLOR_CONVERT_RGB:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image = image)['image']
            
        return image

In [16]:
# 確定是否將應用此增強。機率為 p = 1.0 意味著我們總是從上面應用轉換。
# p = 0 將意味著將忽略轉換塊。
# 0 < p < 1.0 等於每個擴增都具有以一定概率應用的選項。
# OneOf 隨機選取一種增強擴增

def get_test_transforms(count):
    return A.Compose([
        A.Blur(blur_limit = BLUR_LIMIT[count], 
               p = P_BLUR[count]), # 模糊
        A.HorizontalFlip(p = P_HORIZONTALFLIP[count]), # 水平翻轉
        A.VerticalFlip(p = P_VERTICALFLIP[count]), # 垂直翻轉
        A.Flip(p = P_FLIP[count]), # 水平和垂直翻轉
        A.Resize(height = RESIZE_HEIGHT[count], 
                 width = RESIZE_WIDTH[count], 
                 p = P_RESIZE[count]), # 縮放
        A.RandomResizedCrop(height = RANDOMRESIZEDCROP_HEIGHT[count], 
                            width = RANDOMRESIZEDCROP_WIDTH[count], 
                            scale = RANDOMRESIZEDCROP_SCALE[count], 
                            p = P_RANDOMRESIZEDCROP[count]), #隨機縮放剪裁
        A.RandomRotate90(p = P_RANDOMROTATE90[count]), # 隨機旋轉90度
        A.ShiftScaleRotate(shift_limit = SHIFTSCALEROTATE_SHIFT_LIMIT[count], 
                           scale_limit = SHIFTSCALEROTATE_SCALE_LIMIT[count], 
                           rotate_limit = SHIFTSCALEROTATE_ROTATE_LIMIT[count], 
                           p = P_SHIFTSCALEROTATE[count]), # 平移縮放旋轉
        A.ElasticTransform(alpha = ELATICTRANSFORM_ALPHA[count], 
                           sigma = ELATICTRANSFORM_SIGMA[count], 
                           alpha_affine = ELATICTRANSFORM_ALPHA_AFFINE[count], 
                           p = P_ELATICTRANSFORM[count]), # 彈性變換
        A.GridDistortion(num_steps = GRIDDISTORTION_NUM_STEPS[count], 
                         p = P_GRIDDISTORTION[count]), # 網格失真
        A.RandomBrightnessContrast(brightness_limit = RANDOMBRIGHTNESSCONTRAST_BRIGHTNESS_LIMIT[count], 
                                   contrast_limit = RANDOMBRIGHTNESSCONTRAST_CONTRAST_LIMIT[count], 
                                   p = P_RANDOMBRIGHTNESSCONTRAST_CONTRAST[count]), # 隨機亮度對比度
        A.HueSaturationValue(hue_shift_limit = HUESATURATIONVALUE_HUE_SHIFT_LIMIT[count], 
                             sat_shift_limit = HUESATURATIONVALUE_SAT_SHIFT_LIMIT[count], 
                             val_shift_limit = HUESATURATIONVALUE_VAL_SHIFT_LIMIT[count], 
                             p = P_HUESATURATIONVALUE[count]), # 隨機色調飽和度值
        A.CLAHE(clip_limit = CLAHE_CLIP_LIMIT[count], 
                p = P_CLAHE[count]), # 將對比度受限的自適應直方圖均衡化應用於輸入圖像
        A.Cutout(num_holes = COARSEDROPOUT_NUM_HOLES[count], 
                        max_h_size = COARSEDROPOUT_MAX_H_SIZE[count], 
                        max_w_size = COARSEDROPOUT_MAX_W_SIZE[count], 
                        p = P_COARSEDROPOUT[count]), # 隨機在圖像上生成黑色矩形
        A.Normalize(
             mean = NORMALIZE_MEAN[count], 
             std = NORMALIZE_STD[count], 
            max_pixel_value = NORMALIZE_MAX_PIXEL_VALUE[count], 
            p = P_NORMALIZE[count]), # 正規化。
        ToTensorV2(p = P_TOTENSORV2[count]) # 歸一化
    ], p = P_TEST_TRANSFORMS[count])

In [17]:
def prepare_dataloader(count):
    
    test_dataset = MyDataset(test_csv, transforms = get_test_transforms(count))
    
    test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE[count], pin_memory = False, 
                                               shuffle = False)
    return test_loader

In [18]:
# def inference_one_epoch(model, data_loader, device):
#     model.eval()
#     image_preds_all = []
#     # pbar = tqdm(enumerate(data_loader), total=len(data_loader))
#     with torch.no_grad():
#         for step, (imgs) in enumerate(data_loader):
#             imgs = imgs.to(device).float()

#             image_preds = model(imgs)   #output = model(input)
#             image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
#     image_preds_all = np.concatenate(image_preds_all, axis=0)
#     return image_preds_all

In [19]:
def inference_process(count, fold, kf):
    if kf:
        print('Fold %i - image size %i with %s and batch size %i'%(fold+1,IMAGE_SIZE[count],LOAD_MODEL_NAME[count].upper(),BATCH_SIZE[count]))
    else:
        print('Image size %i with %s and batch_size %i'%(IMAGE_SIZE[count],LOAD_MODEL_NAME[count].upper(),BATCH_SIZE[count]))
    
    test_loader = prepare_dataloader(count)
    
    load_model_path = LOAD_MODEL_PATH[count] + LOAD_MODEL_NAME[count] + '.pth'

    build_model(count, load_model_path)
    model = model.to(DEVICE)
    model.eval()
    
    del test_loader
    gc.collect()

In [20]:
def main():
    try:
        print('Inference start')
        since = time.time()
        for count in range(len(LOAD_MODEL_NAME)):
            print('Model {:d}'.format(count+1))
            if FOLD[count] > 1:
                for fold in enumerate(KF.split(np.arange(train_csv.shape[0]))):
                    inference_process(count, fold = fold, kf = True)
            else:
                inference_process(count, fold = 0, kf = False)
        time_elapsed = time.time() - since
        print('Inference complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    except Exception as exception:
        print(exception)
        raise

In [21]:
if __name__ == '__main__':
    main()

Inference start
Model 1
Image size 224 with EFFICIENTNET_V2_B0 and batch_size 256
Error(s) in loading state_dict for EfficientNet:
	Missing key(s) in state_dict: "conv_stem.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "blocks.0.0.conv.weight", "blocks.0.0.bn1.weight", "blocks.0.0.bn1.bias", "blocks.0.0.bn1.running_mean", "blocks.0.0.bn1.running_var", "blocks.1.0.conv_exp.weight", "blocks.1.0.bn1.weight", "blocks.1.0.bn1.bias", "blocks.1.0.bn1.running_mean", "blocks.1.0.bn1.running_var", "blocks.1.0.conv_pwl.weight", "blocks.1.0.bn2.weight", "blocks.1.0.bn2.bias", "blocks.1.0.bn2.running_mean", "blocks.1.0.bn2.running_var", "blocks.1.1.conv_exp.weight", "blocks.1.1.bn1.weight", "blocks.1.1.bn1.bias", "blocks.1.1.bn1.running_mean", "blocks.1.1.bn1.running_var", "blocks.1.1.conv_pwl.weight", "blocks.1.1.bn2.weight", "blocks.1.1.bn2.bias", "blocks.1.1.bn2.running_mean", "blocks.1.1.bn2.running_var", "blocks.2.0.conv_exp.weight", "blocks.2.0.bn1.weight", "blocks

RuntimeError: Error(s) in loading state_dict for EfficientNet:
	Missing key(s) in state_dict: "conv_stem.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "blocks.0.0.conv.weight", "blocks.0.0.bn1.weight", "blocks.0.0.bn1.bias", "blocks.0.0.bn1.running_mean", "blocks.0.0.bn1.running_var", "blocks.1.0.conv_exp.weight", "blocks.1.0.bn1.weight", "blocks.1.0.bn1.bias", "blocks.1.0.bn1.running_mean", "blocks.1.0.bn1.running_var", "blocks.1.0.conv_pwl.weight", "blocks.1.0.bn2.weight", "blocks.1.0.bn2.bias", "blocks.1.0.bn2.running_mean", "blocks.1.0.bn2.running_var", "blocks.1.1.conv_exp.weight", "blocks.1.1.bn1.weight", "blocks.1.1.bn1.bias", "blocks.1.1.bn1.running_mean", "blocks.1.1.bn1.running_var", "blocks.1.1.conv_pwl.weight", "blocks.1.1.bn2.weight", "blocks.1.1.bn2.bias", "blocks.1.1.bn2.running_mean", "blocks.1.1.bn2.running_var", "blocks.2.0.conv_exp.weight", "blocks.2.0.bn1.weight", "blocks.2.0.bn1.bias", "blocks.2.0.bn1.running_mean", "blocks.2.0.bn1.runni

# 7. 待辦事項<a class="anchor" id="7"></a>
[Back to Table of Contents](#0)
1. FOLD
1. TTA
1. ENSEMBLE